In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils
import torch.utils.data as data
import torchvision.models as models
import torchvision.utils as v_utils
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, TextArea
from matplotlib.cbook import get_sample_data
from PIL import ImageFile
import os

In [7]:
ImageFile.LOAD_TRUNCATED_IMAGES = True
image_size = 256
PATH = "D:testWebtoonsData/"

In [8]:
data = datasets.ImageFolder(PATH,transform= transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Grayscale(num_output_channels=3)
        ]))

print(data.class_to_idx)

class_to_idx = data.class_to_idx
idx_to_class = {}
for key, value in enumerate(class_to_idx):
    idx_to_class[key] = value
    
print(idx_to_class)

img_list = []
for i in data.imgs:
    img_list.append(i[0])

#img_list2 = []

#for img in os.listdir('/content/drive/My Drive/dataset/thumnail'):
#    img_list2.append(os.path.join('/content/drive/My Drive/dataset/thumnail',img))
#img_list2.sort()

{'1초': 0, '1학년 9반': 1}
{0: '1초', 1: '1학년 9반'}


In [9]:
resnet = models.resnet50(pretrained=True)

class Resnet(nn.Module):
    def __init__(self):
        super(Resnet,self).__init__()
        self.layer0 = nn.Sequential(*list(resnet.children())[0:1])
        self.layer1 = nn.Sequential(*list(resnet.children())[1:4])
        self.layer2 = nn.Sequential(*list(resnet.children())[4:5])
        self.layer3 = nn.Sequential(*list(resnet.children())[5:6])
        #self.layer4 = nn.Sequential(*list(resnet.children())[6:7])
        #self.layer5 = nn.Sequential(*list(resnet.children())[7:8])

    def forward(self,x):
        out_0 = self.layer0(x)
        out_1 = self.layer1(out_0)
        out_2 = self.layer2(out_1)
        out_3 = self.layer3(out_2)
        #out_4 = self.layer4(out_3)
        #out_5 = self.layer5(out_4)

        return out_0, out_1, out_2, out_3, # out_4, out_5

c:\Users\smhrd\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\smhrd\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\smhrd/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [10]:
class GramMatrix(nn.Module):
    def forward(self, input):
        b,c,h,w = input.size()
        F = input.view(b, c, h*w)
        G = torch.bmm(F, F.transpose(1,2)) 
        return G

class GramMSELoss(nn.Module):
    def forward(self, input, target):
        out = nn.MSELoss()(GramMatrix()(input), target)
        return out

In [12]:
resnet = Resnet()
for param in resnet.parameters():
    param.requires_grad = False

In [14]:
total_arr = []
label_arr = []

for idx,(image,label) in enumerate(data):
    i = image
    i = i.view(-1,i.size()[0],i.size()[1],i.size()[2])

    style_target = list(GramMatrix()(i) for i in resnet(i))

    arr = torch.cat([style_target[0].view(-1),style_target[1].view(-1),style_target[2].view(-1),style_target[3].view(-1)],0)
    gram = arr.data.numpy().reshape(1,-1)

    total_arr.append(gram.reshape(-1))
    label_arr.append(label)

    if idx % 50 == 0 and idx != 0:
        print(f'{idx} images style feature extracted..[{round(idx / len(data), 2) * 100}%]')
print('Image style feature extraction done.')

50 images style feature extracted..[3.0%]
100 images style feature extracted..[6.0%]
150 images style feature extracted..[10.0%]
200 images style feature extracted..[13.0%]
250 images style feature extracted..[16.0%]
300 images style feature extracted..[19.0%]
350 images style feature extracted..[23.0%]
400 images style feature extracted..[26.0%]
450 images style feature extracted..[28.999999999999996%]
500 images style feature extracted..[32.0%]
550 images style feature extracted..[36.0%]
600 images style feature extracted..[39.0%]
650 images style feature extracted..[42.0%]
700 images style feature extracted..[45.0%]
750 images style feature extracted..[49.0%]
800 images style feature extracted..[52.0%]
850 images style feature extracted..[55.00000000000001%]
900 images style feature extracted..[57.99999999999999%]
950 images style feature extracted..[62.0%]
1000 images style feature extracted..[65.0%]
1050 images style feature extracted..[68.0%]
1100 images style feature extracted..

In [15]:
model = TSNE(n_components=2, init='pca',random_state=0, verbose=3, perplexity=100)
result = model.fit_transform(total_arr)

c:\Users\smhrd\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1544 samples in 0.399s...
[t-SNE] Computed neighbors for 1544 samples in 17.866s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1544
[t-SNE] Computed conditional probabilities for sample 1544 / 1544
[t-SNE] Mean sigma: 19858.847902
[t-SNE] Computed conditional probabilities in 0.472s


c:\Users\smhrd\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:982: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


[t-SNE] Iteration 50: error = 46.1192474, gradient norm = 0.0006640 (50 iterations in 0.426s)
[t-SNE] Iteration 100: error = 45.3067818, gradient norm = 0.0003552 (50 iterations in 0.406s)
[t-SNE] Iteration 150: error = 44.7790794, gradient norm = 0.0002431 (50 iterations in 0.369s)
[t-SNE] Iteration 200: error = 44.3901978, gradient norm = 0.0001836 (50 iterations in 0.371s)
[t-SNE] Iteration 250: error = 44.0851669, gradient norm = 0.0001470 (50 iterations in 0.371s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 44.085167
[t-SNE] Iteration 300: error = 1.1828787, gradient norm = 0.0001505 (50 iterations in 0.367s)
[t-SNE] Iteration 350: error = 1.1679806, gradient norm = 0.0001336 (50 iterations in 0.368s)
[t-SNE] Iteration 400: error = 1.1488595, gradient norm = 0.0001148 (50 iterations in 0.372s)
[t-SNE] Iteration 450: error = 1.1290512, gradient norm = 0.0000985 (50 iterations in 0.369s)
[t-SNE] Iteration 500: error = 1.1101335, gradient norm = 0.0000852 (50 

In [ ]:
print(result[:2291][0].mean(), result[:2291][1].mean())
print(label_arr[2291])
print(result[2292:][0].mean(),result[2292:][1].mean())
print(label_arr[2292])

In [21]:
import pandas as pd

result_img_vec = pd.DataFrame({'webtoon_title':[], 'vec_Xmean':[], 'vec_Ymean':[]})
divide_idx = []

for i in range(len(result)):
    if(i == 0):
        divide_idx.append(i)
    if(i == (len(result) - 1)):
        divide_idx.append(len(result))
    if((i != 0) & (label_arr[i-1] != label_arr[i])):
        divide_idx.append(i)

print(divide_idx)

for i in range(len(divide_idx) - 1):
    result_img_vec = result_img_vec.append({'webtoon_title':[idx_to_class[label_arr[divide_idx[i]]]],
                                            'vec_Xmean':[result[divide_idx[i]:divide_idx[i + 1]][0].mean()],
                                            'vec_Ymean':[result[divide_idx[i]:divide_idx[i + 1]][1].mean()]},
                                            ignore_index=True)

result_img_vec.to_csv('D:result_webtoon_vec.csv', encoding='UTF-8')
result_img_vec

[0, 840, 1544]


C:\Users\smhrd\AppData\Local\Temp\ipykernel_7284\854956500.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_img_vec = result_img_vec.append({'webtoon_title':[idx_to_class[label_arr[divide_idx[i]]]],
C:\Users\smhrd\AppData\Local\Temp\ipykernel_7284\854956500.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_img_vec = result_img_vec.append({'webtoon_title':[idx_to_class[label_arr[divide_idx[i]]]],


,webtoon_title,vec_Xmean,vec_Ymean
0,[1초],[30986.615],[34373.504]
1,[1학년 9반],[6509.3223],[40625.094]


In [ ]:
def imscatter(x, y, image, ax=None, zoom=1, show_by_thumnail=False, title='webtoon'):
    if ax is None:
        ax = plt.gca()
    try:
        image = plt.imread(image)
    except TypeError:
        # Likely already an array...
        pass
    im = OffsetImage(image, zoom=zoom)

    # Convert inputs to arrays with at least one dimension.
    x, y = np.atleast_1d(x, y)
    
    artists = []
    for x0, y0 in zip(x, y):
        ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=False)
        

        if show_by_thumnail:
            offsetbox = TextArea(title, minimumdescent=False)
            ac = AnnotationBbox(offsetbox, (x0, y0),
                        xybox=(20, -40),
                        xycoords='data',
                        boxcoords="offset points")
            artists.append(ax.add_artist(ac))
        artists.append(ax.add_artist(ab))

    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return artists

In [ ]:
plt.figure(figsize=(20, 12))

for i in range(len(result)):
    img_path = img_list[i]
    imscatter(result[i,0],result[i,1], image=img_path, zoom=0.2)
plt.show()

In [ ]:
avg_list = []
scatter_x = result[:, 0]
scatter_y = result[:, 1]
group = np.array(label_arr)

for g in np.unique(group):
    i = np.where(group==g)
    x_avg = np.mean(scatter_x[i])
    y_avg = np.mean(scatter_y[i])
    avg_list.append((x_avg, y_avg))

In [ ]:
plt.figure(figsize=(20, 12))

for i in range(len(avg_list)):
    img_path = img_list2[i]
    imscatter(avg_list[i][0],avg_list[i][1], image=img_path,zoom=0.6, show_by_thumnail=True, title=idx_to_class[i])
plt.show()